# 微分可能LUTモデルによるMobileNetライクなMNIST学習

Differentiable LUTモデルでMobileNet風の畳み込み層を形成して、一般的なデータに対してCNNによる回路学習を行います。 

In [ ]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

### データセット

データセットの準備には torchvision を使います

In [ ]:
# configuration
net_name              = 'MnistDifferentiableLutMobileNet'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_cnn'
rtl_module_name       = 'MnistLutCnn'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

bin_mode              = True
frame_modulation_size = 7
epochs                = 4
mini_batch_size       = 64


# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワーク構築

make_conv_layer にて pointwise-depthwise-pointwise の畳み込みを組み合わせた層を作ります。

Convolution2d は指定したモデルを im2col とcol2im で挟み込んで畳み込み層を生成します。

pointwise はフィルタサイズが1x1の通常のConv層です<br>
depthwise は畳み込み内部に持つ DifferentiableLut に同一チャネル内での接続のみを指定して構成します。<br>

In [ ]:
# バイナリ時は BIT型を使えばメモリ削減可能
bin_dtype = bb.DType.BIT if bin_mode else bb.DType.FP32

def make_conv_layer(output_ch, hidden_ch, padding='valid', bin_dtype=bb.DType.BIT):
    return bb.Sequential([
                # input(pointwise)
                bb.Convolution2d(
                    bb.Sequential([
                        bb.DifferentiableLut([hidden_ch*6, 1, 1], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([hidden_ch,   1, 1], connection='serial', bin_dtype=bin_dtype),
                    ]),
                    filter_size=(1, 1),
                    fw_dtype=bin_dtype),

                # hidden(depthwise)
                bb.Convolution2d(
                    bb.Sequential([
                        bb.DifferentiableLut([hidden_ch, 1, 1], connection='depthwise', bin_dtype=bin_dtype),
                    ]),
                    filter_size=(3, 3), padding=padding,
                    fw_dtype=bin_dtype),
                
                # output(pointwise)
                bb.Convolution2d(
                    bb.Sequential([
                        bb.DifferentiableLut([output_ch*6, 1, 1], connection='serial', bin_dtype=bin_dtype),
                        bb.DifferentiableLut([output_ch,   1, 1], connection='serial', bin_dtype=bin_dtype),
                    ]),
                    filter_size=(1, 1),
                    fw_dtype=bin_dtype),
            ])

# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype),
            bb.Sequential([
                make_conv_layer(36, 36, bin_dtype=bin_dtype),  # 28x28 -> 26x26
                make_conv_layer(2*36, 36, bin_dtype=bin_dtype),  # 26x26 -> 24x24
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),  # 24x24 -> 12x12
            ]),
            bb.Sequential([
                make_conv_layer(2*36, 72, bin_dtype=bin_dtype),  # 12x12 -> 10x10
                make_conv_layer(4*36, 72, bin_dtype=bin_dtype),  # 10x10 -> 4x4
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),
            ]),
            bb.Sequential([
                bb.Convolution2d(
                    bb.Sequential([
                        bb.DifferentiableLut([6*128], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([128], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10*6*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(4, 4),
                    fw_dtype=bin_dtype),
            ]),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size, bin_dtype=bin_dtype)
        ])

net.set_input_shape([1, 28, 28])

if bin_mode:
    net.send_command("binary true")

# print(net.get_info())

## 学習実施

学習を行います

In [ ]:
#bb.load_networks(data_path, net)

# learning
loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

for epoch in range(epochs):
    loss.clear()
    metrics.clear()

    # learning
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)

            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            net.backward(dy_buf)

            optimizer.update()

            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    bb.save_networks(data_path, net)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

## RTL(Verilog)変換

FPGA化するために Verilog に変換します。インターフェースはXilinx社のAXI4 Stream Video 仕様(フレームスタートでtuserが立つ)となります。
MaxPooling の単位で画像サイズが縮小されてしまうので、現状、この単位でしか変換できないため3つに分けて出力しています。

In [ ]:
# export verilog
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv0', net[1])
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv1', net[2])
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv2', net[3])

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)